In [ ]:
import csv
import gc
import cv2
from tensorflow import keras
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt
import neptune
from tensorflow.python.client import device_lib
from PIL import Image
import tensorflow as tf
import os
from keras.layers import LeakyReLU
from pathlib import Path
from decouple import config
import matplotlib
from matplotlib import cm
import itertools
from pathlib import Path
from scipy.io import savemat


In [ ]:
# Numerical Case 399
# os.chdir('/aijjeh_odroid_sensors/aidd/data/raw/num/wavefield_dataset2_bottom_out/399_output/')
# 
# images = []
# for i in range(512):
#     data = cv2.imread('%d_flat_shell_Vz_399_500x500bottom.png' % (i+1), 0) 
#     img = cv2.resize(data, (512,512), interpolation=cv2.INTER_CUBIC)
#     images.append(img)
# images = np.asarray(images)    
# images = np.reshape(images, (512, 512, 512, 1))
# images = images.astype('float32')
# images = images / 255.0
# # for j in range(512):    
# #     plt.imshow(images[j])
# #     plt.show()
# #     print(j)
#     
# 
# os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/experimental_cases_set/')
# 
# np.save('GT_Num_output_case_399', images)

In [ ]:
# os.chdir('/aijjeh_odroid_sensors/aidd/data/interim/exp/L3_S4_B/compressed/389286p_512x512/')
# 
# images = []
# for i in range(1,513):
#     data = cv2.imread('frame_%d_389286p_512x512.png' % i, 0) 
#     images.append(data)
# images = np.asarray(images)    
# images = np.reshape(images, (512, 512, 512, 1))
# images = images.astype('float32')
# images = images / 255.0
# # for j in range(512):    
# #     plt.imshow(images[j])
# #     plt.show()
# #     print(j)
#     
# 
# os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/experimental_cases_set/')
# 
# np.save('389286p_na_512x512p_New_exp', images)

In [ ]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
print(device_lib.list_local_devices())

In [ ]:
########################################################################################################################
# Load dataset
########################################################################################################################
def load_dataset():    
    os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/experimental_cases_set/')            
    Full_W_dataset = np.load('389286p_na_512x512p_New_exp.npy')
    print(Full_W_dataset.shape)
    CS_arr = np.load('32x32p_50kHz_5HC_14Vpp_x10_New_exp.npy')
    print(CS_arr.shape)
    x_test = CS_arr
    y_test = Full_W_dataset    
    return x_test, y_test  

In [ ]:
########################################################################################################################
os.chdir('/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Saeed_model/h5_model')
new_dim = 32
CR = (new_dim ** 2) / (69 ** 2)  # Compression Ratio
rescale_factor = int(512 / new_dim)
model_name = 'Sub_pix_latest.h5' 
model = load_model(model_name, compile=False)
model.summary()

In [ ]:
def testing():   
    x_test, y_test = load_dataset()                 
    print(x_test.shape)
    print(y_test.shape)
    
    prediction = model.predict(x_test, batch_size=1)
    prediction = np.asarray(prediction)
    frames = x_test.shape[0]
    
    print(frames)
    for i in range(frames):
        SR_pred = prediction[i].astype('float32')            
        original = x_test[i].astype('float32')
        GT_label_input = y_test[i].astype('float32')
        ############################################################################################################
        plt.figure(figsize=(1, 1), dpi=512)
        plt.gca().set_axis_off()
        plt.axis('off')
        plt.subplots_adjust(left=0.0, bottom=0.0, right=1, top=1.0, wspace=0.0, hspace=0.0)
        plt.margins(0, 0)
        plt.gca().xaxis.set_major_locator(plt.NullLocator())
        plt.gca().yaxis.set_major_locator(plt.NullLocator())
        ############################################################################################################
        SR_pred = SR_pred * 255
        SR_pred= SR_pred.astype(np.uint8)
        # ax1 = plt.subplot(1, 3, 1)
        # ax1.set_title('Original input')
        # ax1.imshow(original)
        # plt.axis('off')

        # ax2 = plt.subplot(1, 4, 2)
        # ax2.set_title('lr input')
        # ax2.imshow(lr_input)
        # plt.axis('off')
 
        
        # ax1.set_title('SR output')        
        plt.imshow(SR_pred)        
        plt.axis('off')
        # ax3 = plt.subplot(1, 3, 3)
        # ax3.set_title('GT')
        # ax3.imshow(GT_label_input)
        # plt.axis('off')            
        
        path = '/home/aijjeh/Desktop/Phd_Projects/compressive_sensing_project/Saeed_model/Saeed_CS_Exp/'
        Path(path).mkdir(parents=True, exist_ok=True)
        os.chdir(path)  
        cv2.imwrite('Saeed_SR_Pred_frame_%d_UNIFORM_MESH.png' % (i+1), SR_pred)
        # plt.savefig('Saeed_SR_Pred_frame_%d_UNIFORM_MESH.png' % (i+1))
        plt.close('all')

In [ ]:
testing()
